In [1]:
from __future__ import print_function, division
#Allows relative imports
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#imports from files
from src.preprocessing import *
from src.VAE_train import *
from src.vautoencoders import *
from src.loss_metrics import *
from src.pickling import *
from src.datasets import *
from vae_cel.vae_cel import * 
from src.preprocessing import * 
from vae_cel.vae_cel_train import test_model
from vae_cel.vae_cel_loss import *
import pandas as pd 
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

#checking gpu status
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))
    
#Plot and stuff
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 200
sns.set_style('darkgrid')

torch.cuda.empty_cache()
# Ignore warnings)
import warnings
warnings.filterwarnings("ignore")
    
%load_ext autoreload
%autoreload 2

Using : cuda


In [2]:
test = pd.read_csv('../training_data_new/mixed_vj_dataset/mixed_vj_test.csv')
test_dataset = test.query('amino_acid.str.len()<=23')[['amino_acid','v_family','j_family']].values

In [2]:
device = 'cpu'
fns = [os.path.join('../output/VAE_CEL_30epochs/',x) for x in os.listdir('../output/VAE_CEL_30epochs/') if ('BEST' in x and '91' in x)]
VAE_VJ50 = VAE_cel(seq_len = 23, aa_dim = 25, latent_dim = 50,
                   act = nn.SELU(), v_dim = 30, j_dim = 2)
tmp = torch.load(fns[1])
VAE_VJ50.load_state_dict(tmp['state_dict'])
VAE_VJ50.to(device)

VAE_VJ100 = VAE_cel(seq_len = 23, aa_dim = 25, latent_dim = 100,
                   act = nn.SELU(), v_dim = 30, j_dim = 2)
tmp = torch.load(fns[0])
VAE_VJ100.load_state_dict(tmp['state_dict'])
VAE_VJ100.to(device);

In [10]:
criterion = VAELoss_cel()
VAE_VJ50.to(device)

#
df1 = test_model(VAE_VJ50, criterion, test_dataset, 2**13, 23, 0.5, 'before', True, False, 'cuda')
df2 = test_model(VAE_VJ100, criterion, test_dataset, 2**13, 23, 0.5, 'before', True, False, 'cuda')
print('VAE50')
display(df1)
print('VAE100')
display(df2)


test set:   0%|          | 0/123 [00:00<?, ?it/s]

test set:   0%|          | 0/123 [00:00<?, ?it/s]

VAE50


,hamming_seq_padded,hamming_sequence,accuracy_V,accuracy_J,Criterion_loss
mean,4.578157,6.883565,0.999742,0.999953,7.179024e-02
var,0.00448,0.009329,0.0,0.0,6.744953e-07


VAE100


,hamming_seq_padded,hamming_sequence,accuracy_V,accuracy_J,Criterion_loss
mean,2.666265,3.944656,0.999879,0.999966,4.330573e-02
var,0.004309,0.008556,0.0,0.0,9.445374e-07


In [ ]:
def plot_latent(df, cols):
    #vars_= [z for z in df.columns if z.startswith('PCA')]
    vars_ = cols
    print(f'VAE latent space representation for {cols[0]}-{cols[-1]}')
    g = sns.PairGrid(df, x_vars = vars_,
                     y_vars = vars_,
                     diag_sharey=False, hue=None)
    g.fig.suptitle(f'VAE latent space representation for {cols[0]}-{cols[-1]}', fontweight='bold')
    g.fig.subplots_adjust(top=0.92)
    g.map_lower(sns.scatterplot,  alpha=0.75)
    g.map_upper(sns.kdeplot)
    g.map_diag(sns.kdeplot)
    g.add_legend()
    
#sns.set_palette("husl", n_colors=4)
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
#vae.to(device)
chks = chunks(range(len(test_dataset)),2**15)
df = pd.DataFrame(columns = (['sequence'] + ['z_'+str(x) for x in list(range(100))]))
VAE_VJ100.eval()

for b in tqdm(list(chks)): 
    with torch.no_grad():
        onehot_tuple = batch_aa_vj(test_dataset[b], 23, 0.5, 'before', 
                                   True, False, 'cpu') 
        z = VAE_VJ100.embed(onehot_tuple)
    df_s = pd.DataFrame(test_dataset[b], columns = ['sequence','v','j'])
    df_z = pd.DataFrame(z, columns = ['z_'+str(x) for x in list(range(100))])
    df = df.append(df_s.join(df_z), ignore_index=True)


chks = list(chunks(df.columns[1:],3))
for ch in tqdm(chks):
    plot_latent(df.sample(frac=0.15), ch)

  0%|          | 0/31 [00:00<?, ?it/s]

In [8]:
z = VAE_VJ100.sample_latent(20)
decoded, sequences, x_v, x_j = VAE_VJ100.reconstruct_latent(z)
tuples = [(a,b,c) for a,b,c in zip(sequences, x_v, x_j)]
tuples

[('CASSQGVTPTEYFVXXXXXXXXX', 6, 1),
 ('GAISQEWEGRGCCVXXXXXXXXX', 10, 1),
 ('CASTEWGEAHTQTFXXXXXXXXX', 18, 1),
 ('CASSPGYRNTQYFXXXXXXXXXX', 1, 0),
 ('CASSQDRQLQVFXXXXXXXXXXX', 5, 1),
 ('CASAQAPNQYFXXXXXXXXXXXX', 13, 1),
 ('CAITFKFGINGGDVVYFXXXXXX', 11, 1),
 ('CASSYGQPVRDQYFXXXXXXXXX', 6, 1),
 ('CASSLEGLEAGGYTCXXXXXXXX', 5, 0),
 ('CAGSTGAGGAEAFFXXXXXXXXX', 6, 0),
 ('CASTDGIAGDQAQHFXXXXXXXX', 5, 0),
 ('CAISGPIGINRGEQFFXXXXXXX', 9, 1),
 ('CASSLEPVTETEQFCXXXXXXXX', 6, 1),
 ('CASSQSGSSASKNNQYFXXXXXX', 4, 1),
 ('CASSLRSQEVFSXXXXXXXXXXX', 3, 1),
 ('CASSQNWVTYYQQFCXXXXXXXX', 3, 1),
 ('CAGFFGLRPYEQFVXXXXXXXXX', 26, 1),
 ('CATNGTKTDPYTFXXXXXXXXXX', 11, 0),
 ('CASSIAIRVAGGELFXXXXXXXX', 4, 1),
 ('CASTLSPQGLSEEQFVXXXXXXX', 10, 1)]

In [61]:
#Quick DeepRC dimension check
N_per_bag = [100, 99, 86, 102] #Fake repertoire with 100, 99, 86, 102 sequences
z = VAE_VJ100.sample_latent(sum(N_per_bag)) # dimension (N_seq, 100)
at = nn.Linear(100,1) # this is the final layer of the "attention network"
attn = at(z) #dim (N, 1), one score per sequence

start_i = 0
results = []
#then here, slice with the number of sequence per bag 
# in order to do the Softmax PER BAG!
for seqs in N_per_bag:
    #Slice & softmax
    attn_slice = torch.softmax(attn[start_i:start_i+seqs], dim=0) 
    z_slice = z[start_i:start_i+seqs]
    weighted_avg = (z_slice*attn_slice).sum(dim=0)
    results.append(weighted_avg)
    start_i += seqs

results = torch.stack(results, dim=0)
results.shape

torch.Size([4, 100])